In [1]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 경고 뜨지 않게 설정
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False

# 데이터 전처리 알고리즘
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# 학습용과 검증용으로 나누는 함수
from sklearn.model_selection import train_test_split

# 교차 검증
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

# 평가함수
# 분류용
from sklearn.metrics import f1_score

# 모델의 최적의 하이퍼 파라미터를 찾기 위한 도구
from sklearn.model_selection import GridSearchCV

# 머신러닝 알고리즘 - 분류
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier

# 학습 모델 저장을 위한 라이브러리
import pickle

### 파일 불러오기 

In [3]:
# 파일 불러오기
pit_data_2023 = pd.read_csv('2023_투구_statiz.csv')
pit_data_2024 = pd.read_csv('2024_투구_statiz.csv')

In [4]:
# 0번째 행 삭제
pit_data_2023 = pit_data_2023.drop(0)
pit_data_2024 = pit_data_2024.drop(0)

In [5]:
# 삭제할 칼럼 리스트
cols_to_drop = ['Rank', 'Team', 'Sort▼', 'RA9', 'rRA9', 'rRA9pf', 'rRA', 'ROE', 'IB']

# 칼럼 삭제하기 
pit_data_2023.drop(cols_to_drop, axis=1, inplace=True)
pit_data_2024.drop(cols_to_drop, axis=1, inplace=True)

In [6]:
# 칼럼명 변경하기 
pit_data_2023.rename(columns={'WAR▼': 'WAR', 'Name': '선수'}, inplace=True)
pit_data_2024.rename(columns={'WAR▼': 'WAR', 'Name': '선수'}, inplace=True)

In [7]:
print(pit_data_2023.columns.tolist())

['선수', 'G', 'GS', 'GR', 'GF', 'CG', 'SHO', 'W', 'L', 'S', 'HD', 'IP', 'ER', 'R', 'TBF', 'H', '2B', '3B', 'HR', 'BB', 'HP', 'SO', 'BK', 'WP', 'ERA', 'FIP', 'WHIP', 'WAR', '팀', 'pid']


In [42]:
display(pit_data_2023)
display(pit_data_2024)

,선수,G,GS,GR,GF,CG,SHO,W,L,S,...,HP,SO,BK,WP,ERA,FIP,WHIP,WAR,팀,pid
1,페냐,32,32,0,0,0,0,11,11,0,...,18,147,4,16,3.6,4.06,1.17,4.28,한화 이글스,15146.0
2,문동주,23,23,0,0,0,0,8,8,0,...,4,95,1,5,3.72,3.65,1.31,3.08,한화 이글스,15013.0
3,산체스,24,24,0,0,0,0,7,8,0,...,6,99,0,3,3.79,3.92,1.3,2.59,한화 이글스,15643.0
4,주현상,55,0,55,9,0,0,2,2,0,...,3,45,0,0,1.96,3.25,0.84,2.44,한화 이글스,11415.0
5,이태양,50,12,38,6,0,0,3,3,0,...,3,72,0,1,3.23,3.6,1.21,1.78,한화 이글스,10609.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288,김동혁,35,2,33,11,0,0,1,7,0,...,6,24,2,0,7.32,4.52,1.75,-0.51,키움 히어로즈,14674.0
289,변시원,4,0,4,0,0,0,0,0,0,...,2,2,0,0,42.43,14.78,4.29,-0.6,키움 히어로즈,10882.0
290,하영민,57,0,57,7,0,0,3,1,0,...,1,51,0,6,4.64,3.86,1.76,-0.7,키움 히어로즈,11222.0
291,양현,54,0,54,12,0,0,0,5,0,...,0,16,1,0,5.05,5.28,1.82,-0.74,키움 히어로즈,10730.0


,선수,G,GS,GR,GF,CG,SHO,W,L,S,...,HP,SO,BK,WP,ERA,FIP,WHIP,WAR,팀,pid
1,류현진,28,28,0,0,0,0,10,8,0,...,3,135,0,1,3.87,3.67,1.36,4.44,한화 이글스,10590.0
2,주현상,65,0,65,51,0,0,8,4,23,...,1,64,0,1,2.65,3.85,0.84,3.13,한화 이글스,11415.0
3,와이스,16,16,0,0,0,0,5,5,0,...,3,98,0,6,3.73,3.65,1.16,2.92,한화 이글스,16153.0
4,문동주,21,21,0,0,0,0,7,7,0,...,4,96,1,8,5.17,4.71,1.67,1.77,한화 이글스,15013.0
5,바리아,20,19,1,0,0,0,6,7,0,...,0,83,0,6,5.15,3.85,1.42,1.69,한화 이글스,16150.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,전준표,18,5,13,4,0,0,2,4,0,...,5,18,0,4,6.83,7.09,1.99,-0.35,키움 히어로즈,16122.0
290,오석주,17,0,17,7,0,0,1,1,0,...,3,16,0,2,11.12,4.4,2.29,-0.5,키움 히어로즈,12860.0
291,조영건,25,3,22,3,0,0,2,1,0,...,1,31,0,3,8.01,6.01,1.91,-0.55,키움 히어로즈,14132.0
292,윤석원,11,1,10,3,0,0,0,1,0,...,2,1,1,3,11.42,7.72,2.19,-0.66,키움 히어로즈,15071.0


In [9]:
# 1. 2023, 2024 각 데이터에 연도 prefix 붙이기 (pid만 예외)
pit_23 = pit_data_2023.add_prefix('2023_')
pit_24 = pit_data_2024.add_prefix('2024_')

# 2. pid 컬럼명 복원 (prefix 제거)
pit_23.rename(columns={'2023_pid': 'pid'}, inplace=True)
pit_24.rename(columns={'2024_pid': 'pid'}, inplace=True)

# 3. pid 기준 inner 병합 (두 해 모두 있는 선수만)
pit_23_24 = pd.merge(pit_23, pit_24, on='pid', how='inner')

In [40]:
# 결측치 0으로 채우기 
pit_23_24 = pit_23_24.fillna(0)

pit_23_24

,2023_선수,2023_G,2023_GS,2023_GR,2023_GF,2023_CG,2023_SHO,2023_W,2023_L,2023_S,...,2024_BB,2024_HP,2024_SO,2024_BK,2024_WP,2024_ERA,2024_FIP,2024_WHIP,2024_WAR,2024_팀
0,페냐,32,32,0,0,0,0,11,11,0,...,20,0,29,0,1,6.27,5.83,1.63,0.26,한화 이글스
1,문동주,23,23,0,0,0,0,8,8,0,...,38,4,96,1,8,5.17,4.71,1.67,1.77,한화 이글스
2,산체스,24,24,0,0,0,0,7,8,0,...,21,10,56,1,2,4.22,3.82,1.52,1.48,한화 이글스
3,주현상,55,0,55,9,0,0,2,2,0,...,8,1,64,0,1,2.65,3.85,0.84,3.13,한화 이글스
4,이태양,50,12,38,6,0,0,3,3,0,...,2,0,2,0,0,11.57,9.43,2.04,-0.5,한화 이글스
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,김동규,3,1,2,0,0,0,0,1,0,...,2,0,0,0,0,54.0,22.8,9.0,-0.07,키움 히어로즈
200,김동혁,35,2,33,11,0,0,1,7,0,...,10,2,9,0,0,6.85,8.59,1.66,-0.72,키움 히어로즈
201,하영민,57,0,57,7,0,0,3,1,0,...,58,5,101,0,7,4.37,4.45,1.5,3.25,키움 히어로즈
202,양현,54,0,54,12,0,0,0,5,0,...,5,1,10,0,0,7.62,7.37,2.0,0.02,삼성 라이온즈


In [36]:
# 모든 값이 0인 컬럼만 찾아서 삭제
cols_all_zero = [col for col in pit_23_24.columns if (pit_23_24[col] == 0).all()]
pit_23_24 = pit_23_24.drop(columns=cols_all_zero)

print("제거된 칼럼:", cols_all_zero)

제거된 칼럼: []


In [38]:
pit_23_24.to_csv('투수 스탯 데이터.csv', index=False)